# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: boxcovering

Variabili utili per testare il funzionamento


In [5]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


In [6]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Versione iniziale di boxcovering

boxcovering calcola quali bounding box si intersecano tra loro.

In [9]:
function boxcovering(bboxes, index, tree)
	covers = [[] for k=1:length(bboxes)]
	for (i,boundingbox) in enumerate(bboxes)
		extent = bboxes[i][index,:]
		iterator = IntervalTrees.intersect(tree, tuple(extent...))
		for x in iterator
			append!(covers[i],x.value)
		end
	end
	return covers
end

boxcovering (generic function with 1 method)

In [18]:
@btime boxcovering(bb, 1, t)   #8.936 μs

  8.936 μs (86 allocations: 4.67 KiB)


5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

boxcovering è type stable ma la variabile covers è un array di Any. Si procede tipizzando covers e dividendo la funzione in microtask.

### Versione modificata di boxcovering

In [10]:
function boxcoveringMOD(bboxes, index, tree)
    covers = [[zero(eltype(Int64))] for k=1:length(bboxes)]		#zero(eltype(Int64)) serve per rendere covers type stable
    @threads for (i,boundingbox) in collect(enumerate(bboxes))
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        addIntersection(covers, i, iterator)
    end
    return covers
end

boxcoveringMOD (generic function with 1 method)

In [17]:
cov = boxcoveringMOD(bb, 1, t)

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [12]:
ex = bb[1][1,:]

2-element Vector{Float64}:
 0.0
 1.0

In [16]:
it = IntervalTrees.intersect(t, tuple(ex...))


IntervalTrees.IntervalIntersectionIterator{typeof(IntervalTrees.true_cmp), Float64, IntervalValue{Float64, Array}, 64}(IntervalTrees.true_cmp, IntervalTrees.Intersection{Float64, IntervalValue{Float64, Array}, 64}(0, #undef), IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}, Interval{Float64}
(0.0,1.0))


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]
(1.0,1.0) => [2]

In [19]:
@btime boxcoveringMOD(bb, 1, t)    #4.499 μs

  4.499 μs (82 allocations: 3.02 KiB)


5-element Vector{Vector{Any}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [20]:
@benchmark boxcovering(bb, 1, t)   

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   8.885 μs …  3.242 ms  ┊ GC (min … max): 0.00% … 97.68%
 Time  (median):      9.505 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.075 μs ± 73.341 μs  ┊ GC (mean ± σ):  4.83% ±  1.70%

  █▃▁▁▁▂▄▄▃▁                                                  ▁
  █████████████▇▇█▇▇▇▆▆▆▆▆▆▅▆▆▅▅▆▆▅▅▄▄▃▄▄▄▃▃▃▄▃▄▄▄▃▅▄▄▄▂▃▄▅▃▃ █
  8.88 μs      Histogram: log(frequency) by time      82.5 μs <

 Memory estimate: 4.67 KiB, allocs estimate: 86.

funzione modificata:

In [21]:
@benchmark boxcoveringMOD(bb, 1, t)   

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):   4.505 μs …  1.604 ms  ┊ GC (min … max): 0.00% … 99.08%
 Time  (median):      4.749 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.098 μs ± 41.917 μs  ┊ GC (mean ± σ):  4.43% ±  1.98%

  █▂▅▅▂▁▂▁                                                    ▁
  ███████████▇▇▆▇▇▆▆▅▆▅▅▅▄▅▅▄▄▅▃▁▄▄▁▄▅▄▄▁▄▃▃▃▄▄▄▃▃▄▁▁▁▃▃▄▃▁▄▄ █
  4.51 μs      Histogram: log(frequency) by time      98.6 μs <

 Memory estimate: 3.02 KiB, allocs estimate: 82.

# FUNZIONI AGGIUNTIVE

### addIntersection

addIntersection aggiunge in 'covers' in i-esima posizione tutti i bounding box che intersecano l'i-esimo bounding box



In [13]:
function addIntersection(covers::Array{Array{Int64,1},1}, i::Int64, iterator)
    splice!(covers[i],1)		#splice serve a togliere gli zeri iniziali all'interno di covers
    @threads for x in collect(iterator)
        append!(covers[i],x.value)
    end
end

addIntersection (generic function with 1 method)

### CreateIntervalTree

funzione che crea un albero di supporto per la funzione principale **boxcovering**; nel particolare dato un dizionario ordinato crea un intervalTrees cioè una struttura dati che contiene intervalli e che consente di trovare in modo efficiente tutti gli intervalli che si sovrappongono a un determinato intervallo o punto.

In [14]:
function createIntervalTree(boxdict::AbstractDict{Array{Float64,1},Array{Int64,1}})
    tree = IntervalTrees.IntervalMap{Float64,Array}()
    for (key, boxset) in boxdict
        tree[tuple(key...)] = boxset
    end
    return tree
end

createIntervalTree (generic function with 1 method)

In [15]:
t=createIntervalTree(dict)  #creazione dell'albero t

IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]
(1.0,1.0) => [2]

## TEST

In [ ]:
@testset "boxcovering Tests" begin
    V,(VV,EV,FV,CV) = Lar.cuboidGrid([2,2,2],true)
    W,_ = Lar.apply(Lar.r(1,1,pi/6),(V,[VV,EV,FV,CV]))
    cellpoints = [ W[:,EV[k]]::Lar.Points for k=1:length(EV) ]
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
    dict = Lar.coordintervals(1,bboxes)
    @test typeof(dict) == OrderedDict{Array{Float64,1},Array{Int64,1}}
    @test length(Lar.coordintervals(1,bboxes)) == 54
    @test length(Lar.coordintervals(2,bboxes)) == 54
    @test length(Lar.coordintervals(3,bboxes)) == 54

    V,(VV,EV,FV) = Lar.cuboidGrid([2,1],true)
    cellpoints = [ V[:,EV[k]]::Lar.Points for k=1:length(EV) ]
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
    @test bboxes == [[0.0 0.0; 0.0 1.0],
    [1.0 1.0; 0.0 1.0],
    [2.0 2.0; 0.0 1.0],
    [0.0 1.0; 0.0 0.0],
    [0.0 1.0; 1.0 1.0],
    [1.0 2.0; 0.0 0.0],
    [1.0 2.0; 1.0 1.0]]
    xboxdict = Dict(
     [0.0, 0.0] => [1],
     [1.0, 1.0] => [2],
     [2.0, 2.0] => [3],
     [0.0, 1.0] => [4, 5],
     [1.0, 2.0] => [6, 7])
    @test xboxdict == Lar.coordintervals(1,bboxes)
    xs = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in xboxdict
        xs[tuple(key...)] = boxset
    end
   @test typeof(xs) ==
    IntervalTrees.IntervalBTree{Float64,
    IntervalValue{Float64,Array},64}
end

![test di boxcovering](https://github.com/MarcoCap13/LAR-SPLITTING-2D-5.b-/blob/main/docs/test/boxcovering_test.png?raw=true)